### **0. 실습 1: Logistic Regression model 만들기**
[관련 링크](https://github.com/Artinto/Python_and_AI_Study/tree/main/Week7/%5B%EC%8B%A4%EC%8A%B5%5DLogistic_Regression)




### **1. 필요한 모듈 선언하기**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

### **2. Device 선언 및 랜덤 시드 고정**

2-1) device 선언

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


2-2) 랜덤 시드 고정

In [ ]:
torch.manual_seed(777)
if device=='cuda':
  torch.cuda.manual_seed_all(777)
np.random.seed(777)

### **3. 데이터셋 불러오기**

In [ ]:
dataset = np.loadtxt('/content/drive/MyDrive/dataset/diabetes.csv', delimiter=',', dtype=np.float32) 
np.random.shuffle(dataset)

3-1) 데이터 분류하기

In [ ]:
x_train = dataset[:650,:-1]   # train set : 650 개
y_train = dataset[:650,[-1]]

x_test = dataset[650:,:-1]    # test set: 109 개
y_test = dataset[650:,[-1]]


3-2) ndarray to tensor

In [ ]:
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test)

### **4. 모델 선언**

4-1) Model 선언

In [ ]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [ ]:
model = BinaryClassifier()
model = model.to(device)

### **5. 학습 시작**

5-1) optimizer 선언

In [ ]:
lr=1
optimizer = optim.SGD(model.parameters(), lr=lr)

5-2) 학습

In [ ]:
nb_epochs = 1000

In [ ]:
print('************** TRAIN ************** \n')

for epoch in range(nb_epochs+1):
  x_train = x_train.to(device)
  y_train = y_train.to(device)
  hypothesis = model(x_train)
  cost = F.binary_cross_entropy(hypothesis, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch%100 == 0:

    print('Epoch: {:5d}/{} \t Cost: {:.6f} \t'.format(epoch, nb_epochs,cost.item()))

************** TRAIN ************** 

Epoch:     0/1000 	 Cost: 0.679650 	
Epoch:   100/1000 	 Cost: 0.476757 	
Epoch:   200/1000 	 Cost: 0.468684 	
Epoch:   300/1000 	 Cost: 0.466631 	
Epoch:   400/1000 	 Cost: 0.465789 	
Epoch:   500/1000 	 Cost: 0.465393 	
Epoch:   600/1000 	 Cost: 0.465199 	
Epoch:   700/1000 	 Cost: 0.465102 	
Epoch:   800/1000 	 Cost: 0.465053 	
Epoch:   900/1000 	 Cost: 0.465028 	
Epoch:  1000/1000 	 Cost: 0.465016 	


### **6. Test**

In [ ]:
with torch.no_grad():
  x_test = x_test.to(device)
  y_test = y_test.to(device)
  prediction = model(x_test)
  prediction = prediction >= torch.FloatTensor([0.5])
  correct_prediction = prediction.float() == y_test
  accuracy = correct_prediction.sum().item() / len(correct_prediction)
  print('Test Accuracy: {:2.2f}%'.format(accuracy*100))

Test Accuracy: 74.31%
